
本程式碼請搭配文章「[在 Colab 上利用 Yolov3 框架和自有標住資料來訓練自己的物件辨識系統](http://bit.ly/2OAfpq0)」 一起服用。

其實原意是利用圖片辨識手遊的畫面，來幫我自動玩手遊練功。在找資料的時候，很多人都推薦 Yolov3，優點是非常快速，剛好利用在畫面變動巨大的手遊上。但使用遊戲畫面，可能只有我自己看得懂。所以這篇文章我用最普遍的貓和狗當作範例。能訓練貓和狗就能訓練自己其他的資料了。

# 這篇文章會教你
在此範例中，我們將會訓練一個能偵測圖片中貓和狗的模型。你可以自由替換資料集，偵測自訂的物體。

除此之外，還有：
* 利用 Colab 128G RAM GPU 來訓練你的 Yolo3 模型
* 掛載 Google Drive 檔案到 Colab 檔案系統中
* 將 PASCAL VOC 標籤格式轉換成 Yolo 用的標籤格式
* 產生 Yolo 訓練需要的 cfg 設定檔案
* 將訓練後的 weight 檔案同步至 Google Drive 中，避免遺失
* 如何利用 weight 檔案來辨識圖片中的內容

# 事前準備

在開始前，你需要先編譯好 Darknet 執行檔案。你可以參考我的另外一篇文章「[如何在 Colab 安裝 Darknet 框架訓練 YOLO v3 物件辨識並且最佳化 Colab 的訓練流程](http://bit.ly/33XjcEu)」，文章中會將編譯好的 Darknet 執行檔案放到 Google Drive目錄下。本文章將會利用這個執行檔案進行訓練。




---




## 將 Google Drive 掛載到 Colab 目錄下

掛載 Google Drive 的好處是不用每次都手動上傳或下載檔案，而且還能讓訓練好的模型檔案自動保存到 Google Drive。這樣就不會因為 Colab 中斷後就必須從頭訓練。

In [63]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [64]:
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'
!git clone https://github.com/AlexeyAB/darknet/

fatal: destination path 'darknet' already exists and is not an empty directory.


In [65]:
!apt install gcc-5 g++-5 -y
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc 
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++

Reading package lists... Done
Building dependency tree       
Reading state information... Done
g++-5 is already the newest version (5.5.0-12ubuntu1).
gcc-5 is already the newest version (5.5.0-12ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
ln: failed to create symbolic link '/usr/local/cuda/bin/gcc': File exists
ln: failed to create symbolic link '/usr/local/cuda/bin/g++': File exists


In [66]:
%cd darknet
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!make

[Errno 20] Not a directory: 'darknet'
/content/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:909:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -c ./src/http_stream.cpp -o obj/http_stream.o
In file included from ./src/

In [0]:
# !wget https://pjreddie.com/media/files/yolov3.weights
# !chmod a+x ./darknet

In [0]:
# from google.colab import files
# uploaded = files.upload()

In [69]:
!pwd

/content/darknet


In [0]:
!cp /content/gdrive/My\ Drive/train_yolo_with_custom_dataset_on_colab_101/weights/yolov3_last.weights /content/darknet

In [0]:
!cp /content/gdrive/My\ Drive/train_yolo_with_custom_dataset_on_colab_101/cfg/obj.data /content/darknet

In [0]:
!cp /content/gdrive/My\ Drive/train_yolo_with_custom_dataset_on_colab_101/cfg/yolov3.cfg /content/darknet

In [73]:
%cd ..

/content


In [74]:
!mkdir app

mkdir: cannot create directory ‘app’: File exists


In [75]:
%cd app
!mkdir cfg
!cp /content/gdrive/My\ Drive/train_yolo_with_custom_dataset_on_colab_101/cfg/obj.names /content/app/cfg

/content/app
mkdir: cannot create directory ‘cfg’: File exists


In [76]:
%cd ..

/content


In [77]:
%cd darknet

/content/darknet


In [0]:
#videopth = "/content/gdrive/My\ Drive/Final_Project/train_yolo_with_custom_dataset_on_colab_101"
videopth = "/content/gdrive/My\ Drive/Final_Project"

In [81]:
!./darknet detector demo obj.data yolov3.cfg yolov3_last.weights -dont_show {videopth}/Kangaroo.mp4 -out_filename Kangaroo.avi
# !./darknet detector demo obj.data yolov3.cfg yolov3_last.weights -dont_show {videopth}/Kangaroo.mp4 -i 0 -ext_output Kangaroo.avi

Streaming output truncated to the last 5000 lines.

kangaroo: 71% 
kangaroo: 34% 

FPS:40.1 	 AVG_FPS:39.7

 cvWriteFrame 
Objects:

kangaroo: 73% 
kangaroo: 34% 

FPS:40.2 	 AVG_FPS:39.7

 cvWriteFrame 
Objects:

kangaroo: 73% 
kangaroo: 34% 

FPS:40.1 	 AVG_FPS:39.7

 cvWriteFrame 
Objects:

kangaroo: 92% 
kangaroo: 39% 

FPS:40.2 	 AVG_FPS:39.7

 cvWriteFrame 
Objects:

kangaroo: 79% 
kangaroo: 34% 

FPS:40.2 	 AVG_FPS:39.7

 cvWriteFrame 
Objects:

kangaroo: 96% 
kangaroo: 54% 

FPS:40.2 	 AVG_FPS:39.7

 cvWriteFrame 
Objects:


FPS:39.5 	 AVG_FPS:39.7

 cvWriteFrame 
Objects:


FPS:39.5 	 AVG_FPS:39.7

 cvWriteFrame 
Objects:

kangaroo: 41% 

FPS:39.7 	 AVG_FPS:39.7

 cvWriteFrame 
Objects:


FPS:41.3 	 AVG_FPS:39.7

 cvWriteFrame 
Objects:


FPS:39.9 	 AVG_FPS:39.7

 cvWriteFrame 
Objects:


FPS:39.8 	 AVG_FPS:39.7

 cvWriteFrame 
Objects:


FPS:39.9 	 AVG_FPS:39.7

 cvWriteFrame 
Objects:


FPS:40.2 	 AVG_FPS:39.7

 cvWriteFrame 
Objects:


FPS:39.9 	 AVG_FPS:39.7

 cvWriteFrame

In [82]:
from google.colab import files
downloaded = files.download('Kangaroo.avi')

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 52584, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

In [86]:
!./darknet detector demo obj.data yolov3.cfg yolov3_last.weights -dont_show {videopth}/Raccoon.mp4 -out_filename Raccoon.avi

Streaming output truncated to the last 5000 lines.
raccoon: 62% 

FPS:39.6 	 AVG_FPS:41.0

 cvWriteFrame 
Objects:

raccoon: 87% 

FPS:39.3 	 AVG_FPS:41.0

 cvWriteFrame 
Objects:

raccoon: 96% 

FPS:39.4 	 AVG_FPS:41.0

 cvWriteFrame 
Objects:

raccoon: 88% 

FPS:39.5 	 AVG_FPS:41.0

 cvWriteFrame 
Objects:

raccoon: 94% 

FPS:39.6 	 AVG_FPS:41.0

 cvWriteFrame 
Objects:

raccoon: 89% 

FPS:39.5 	 AVG_FPS:41.0

 cvWriteFrame 
Objects:

raccoon: 88% 

FPS:39.6 	 AVG_FPS:41.0

 cvWriteFrame 
Objects:

raccoon: 93% 

FPS:39.7 	 AVG_FPS:41.0

 cvWriteFrame 
Objects:

raccoon: 79% 

FPS:39.9 	 AVG_FPS:41.0

 cvWriteFrame 
Objects:

raccoon: 93% 

FPS:40.6 	 AVG_FPS:41.0

 cvWriteFrame 
Objects:

raccoon: 87% 

FPS:39.9 	 AVG_FPS:41.0

 cvWriteFrame 
Objects:

raccoon: 96% 

FPS:39.9 	 AVG_FPS:41.0

 cvWriteFrame 
Objects:

raccoon: 98% 

FPS:40.0 	 AVG_FPS:41.0

 cvWriteFrame 
Objects:

raccoon: 98% 

FPS:39.6 	 AVG_FPS:41.0

 cvWriteFrame 
Objects:

raccoon: 99% 

FPS:39.7 	 AVG_FPS:41.0


In [87]:
from google.colab import files
downloaded2 = files.download('Raccoon.avi')

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 52394, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil